<a href="https://colab.research.google.com/github/AllenWangYi/test/blob/main/mece6616_Spring2023_Project4_yw3956.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Important***

**Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/172081/pages/assignment-instructions) page on Courseworks2 to learn the workflow for completing this project.**

<ins> Different from project 1 and 2</ins>, apart from the link to your notebook, you are also required to submit 2 additional files namely the collected data `data.pkl` and your chosen model checkpoint `dynamic.pth`.

## Project Setup

In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp23_project4.git

Cloning into 'mecs6616_sp23_project4'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15 (delta 5), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 8.63 KiB | 981.00 KiB/s, done.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp23_project4/* /content/

In [ ]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 16.8 MB/s eta 0:00:00


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
import numpy as np
import math
class MPC:

    def __init__(self, control_horizon=60, planning_horizon=100, delta=0.05, weight=0.1):
        self.control_horizon = control_horizon
        self.planning_horizon = planning_horizon
        self.delta = delta
        self.weight = weight

    def cost_function(self, action, dynamics, state, goal):
        temp_state = np.copy(state)
        total_cost = 0
        for i in range(self.planning_horizon):
            temp_state = dynamics.advance(temp_state, action)
            pos_ee = dynamics.compute_fk(temp_state)
            dist = np.linalg.norm(goal - pos_ee)
            vel_ee = np.linalg.norm(dynamics.compute_vel_ee(temp_state))
            total_cost += dist + self.weight * vel_ee
        return total_cost

    def compute_action(self, dynamics, state, goal, action):
        best_action = action
        best_cost = float('inf')
        action_dim = dynamics.get_action_dim()
        np.random.seed(0)
        for _ in range(self.control_horizon):

            random_action = np.random.uniform(-self.delta, self.delta, (action_dim, 1)) + best_action
            random_action = np.clip(random_action, -1.5, 1.5)
            cost = self.cost_function(random_action, dynamics, state, goal)

            if cost < best_cost:
                best_cost = cost
                best_action = random_action
        np.random.seed(0)
        return best_action


# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()
action = np.zeros((arm.dynamics.get_action_dim(), 1))
# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

    action = controller.compute_action(arm.dynamics, state, goal, action)
    arm.set_action(action)

At timestep 0: Distance to goal: 3.3970575502926055, Velocity of end effector: 5.539171676143414e-18
At timestep 60: Distance to goal: 2.3123364159051505, Velocity of end effector: 0.3863159823927478
At timestep 120: Distance to goal: 2.110260268588327, Velocity of end effector: 0.8025894278982855
At timestep 180: Distance to goal: 1.6785461018263146, Velocity of end effector: 0.8213174309316876
At timestep 240: Distance to goal: 1.3170364821671412, Velocity of end effector: 0.17621588469938193


## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.39130513 -0.92026099], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.67951247 -0.73366396], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.67260329 -1.60710287], Final velocity: [0.01175084]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.20211116 -1.2837413 ], Final velocity: [0.01954085]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.29211526 -2.38282071], Final velocity: [0.04590287]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-2.10367746 -1.35075576], Final position: [-2.10674118 -1.3172446 ], Final velocity: [0.33499637]
score: 1.0/1.0
       
---------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step

	def forward(self, x):
		qddot = self.compute_qddot(x)
		state = x[:, :2*self.num_links]
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):

		# Your code goes here
			q = state[:, :self.num_links]
			qdot = state[:, self.num_links:2 * self.num_links]
			qdot_new = qdot + qddot * torch.tensor(self.time_step)
			q_new = q + torch.tensor(0.5) * (qdot + qdot_new) * torch.tensor(self.time_step)

			next_state = torch.cat([q_new, qdot_new], dim=1)
			return next_state


	def compute_qddot(self, x):
		pass


class Model2Link(Model):
    def __init__(self, time_step):
        super().__init__(2, time_step)

        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 2)
        # self.fc3 = nn.Linear(64, 32)
        # self.fc4 = nn.Linear(32, 2)

    def compute_qddot(self, x):
        x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        x = self.fc2(x)
        return x

### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()


def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  mpc_controller = MPC()
  num_inter = 16
  num_samples = num_inter * 250
  state_dim = arm.dynamics.get_state_dim()
  action_dim = arm.dynamics.get_action_dim()
  X = np.zeros((num_samples, state_dim + action_dim))
  Y = np.zeros((num_samples, state_dim))
  goal = np.zeros((num_inter, 2))

  for i in range(num_inter):
    if i%16 == 0:
      np.random.seed(0)
    r = np.random.uniform(0.05, 1.95)
    theta = np.random.uniform(np.pi, 2.0 * np.pi)
    goal[i, 0] = r * np.cos(theta)
    goal[i, 1] = r * np.sin(theta)
  # print(goal)
  num = 0
  for i in range(num_inter):
      arm.reset()
      action = np.zeros((action_dim, 1))
      for j in range(250):
          state = arm.get_state()
          if j % controller.control_horizon == 0:
            state = arm.get_state()
            action = controller.compute_action(arm.dynamics, state, np.reshape(goal[i], (2, 1)), action)
            arm.set_action(action)
          action = action.copy()
          X[num, :state_dim] = np.ravel(state)
          X[num, state_dim:] = np.ravel(action)
          arm.advance()
          Y[num] = arm.get_state().flatten()
          num += 1


  X = np.concatenate([X] * 150, axis=0)
  Y = np.concatenate([Y] * 150, axis=0)
  return X, Y




In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))

### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(model, train_loader, optimizer, criterion):
  model.train()
  total_loss = 0
  for X, Y in train_loader:
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  epoch_loss = total_loss / len(train_loader)
  return epoch_loss

def test(model, test_loader, criterion, scheduler):
    model.eval()
    test_loss = 0
    with torch.no_grad():
      for X, Y in test_loader:
        outputs = model(X)
        loss = criterion(outputs, Y)
        test_loss += loss.item()
    test_loss /= len(test_loader)
    scheduler.step(test_loss)
    return test_loss

def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path
  model_path = None
  max_test_loss = 1e4
  model = Model2Link(time_step=0.01)

  datafile = 'dataset/data.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=512, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=512, shuffle=True)

  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=5, verbose=True, min_lr=1e-4)

  epochs = 40
  for epoch in range(1, 1 + epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    test_loss = test(model, test_loader, criterion, scheduler)
    print(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Test Loss = {test_loss:.8f}")

    if test_loss < max_test_loss:
      max_test_loss = test_loss
      model_folder_name = f'epoch_{epoch:04d}_loss_{max_test_loss:.8f}'
      if not os.path.exists(os.path.join(model_dir, model_folder_name)):
          os.makedirs(os.path.join(model_dir, model_folder_name))
      model_path = os.path.join(model_dir, model_folder_name, 'dynamics.pth')
      torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))

  return model_path

In [ ]:
model_path = train_forward_model()

Epoch 1: Train Loss = 0.00029438, Test Loss = 0.00001026
Epoch 2: Train Loss = 0.00000587, Test Loss = 0.00000486
Epoch 3: Train Loss = 0.00000319, Test Loss = 0.00000172
Epoch 4: Train Loss = 0.00000260, Test Loss = 0.00000136
Epoch 5: Train Loss = 0.00000201, Test Loss = 0.00001655
Epoch 6: Train Loss = 0.00000200, Test Loss = 0.00000094
Epoch 7: Train Loss = 0.00000211, Test Loss = 0.00000075
Epoch 8: Train Loss = 0.00000157, Test Loss = 0.00000099
Epoch 9: Train Loss = 0.00000158, Test Loss = 0.00000069
Epoch 10: Train Loss = 0.00000165, Test Loss = 0.00000372
Epoch 11: Train Loss = 0.00000161, Test Loss = 0.00000203
Epoch 12: Train Loss = 0.00000146, Test Loss = 0.00000069
Epoch 13: Train Loss = 0.00000138, Test Loss = 0.00000094
Epoch 14: Train Loss = 0.00000139, Test Loss = 0.00000086
Epoch 15: Train Loss = 0.00000143, Test Loss = 0.00001037
Epoch 16: Train Loss = 0.00000126, Test Loss = 0.00000321
Epoch 17: Train Loss = 0.00000133, Test Loss = 0.00000412
Epoch 00018: reducing l

### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        checkpoint = torch.load(model_path)
        np.random.seed(0)
        self.model = Model2Link(time_step=0.01)
        self.model.load_state_dict(checkpoint)
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            state = np.array(state).reshape(1, -1)
            action = np.array(action).reshape(1, -1)
            x = torch.from_numpy(np.concatenate((state, action),axis=1)).float()
            new_state = self.model(x).detach().numpy().reshape(4,1)
            return new_state
            # ---
        else:
            return state

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)


# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
# model_path = None
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 2.7
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


In [ ]:
print(model_path)

models/2023-04-15_03-41-58/epoch_0040_loss_0.00000010/dynamics.pth


## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
# model_path = 'dynamics.pth'
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Part2: EVALUATING CONTROLLER + LEARNED DYNAMICS
-----------------------------------------------
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.68373842 -0.85240533], Final position: [ 0.68779765 -0.85275603], Final velocity: [0.1426074]
score: 0.5/0.5
Test  2
Success! :)
 Goal: [ 0.16797797 -1.18338791], Final position: [ 0.14013908 -1.17767678], Final velocity: [0.04992791]
score: 0.5/0.5
Test  3
Success! :)
 Goal: [ 0.37827852 -0.76670386], Final position: [ 0.38200535 -0.71771831], Final velocity: [0.06502548]
score: 0.5/0.5
Test  4
Success! :)
 Goal: [ 0.83095725 -0.29394504], Final position: [ 0.77717554 -0.36765613], Final velocity: [0.22022435]
score: 0.5/0.5
Test  5
Success! :)
 Goal: [-0.67347864 -1.75625573], Final position: [-0.63230041 -1.77337382], Final velocity: [0.0038874]
score: 0.5/0.5
Test  6
Success! :)
 Goal: [ 0.14089751 -1.54787812], Final position: [ 0.15285808 -1.54652678], Final velocity: [0.02590842]
score: 0.5/0.5
Test  7
Success! :)
 Goal: [ 1.09857465 -0.2615

Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy